In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
print(device)

cpu


In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor())

training_data_len = len(training_data)
training_data, validation_data = random_split(training_data,[int(5/6*training_data_len),int(training_data_len/6)])

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor())

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size: int):
        super().__init__()
        self.input_size = input_size

        self.linear_relu_stack = nn.Sequential(  # Layer 2..n, einfache Netzwerk übereinander (Sequential)
            nn.Flatten(),
            nn.Linear(self.input_size, 128),  # Input 28x28, Output = 128
            nn.ReLU(),
            nn.Linear(128, 10),  # Input 128, Output = 10 / Predictions
            nn.Softmax())

    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x


model = NeuralNetwork(input_size=28 * 28).to(device)  # Angabe wo das ausgeführt werden soll
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=10, bias=True)
    (4): Softmax(dim=None)
  )
)


In [5]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        #if X dann trainiere auf model 2
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [6]:
def test_validate(dataloader, model, loss_fn):
    """
    Returns accuracy, average loss
    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        loss /= num_batches
        correct /= size
        accuracy = 100 * correct
    return  accuracy, loss

In [7]:
loss_fn = nn.CrossEntropyLoss()  # MSE

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # step, update von params = params - lr * grad

batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

for epoch in range(10):
    print(f"Epoch {epoch}\n-------------------------------")
    train(dataloader=train_dataloader, model=model, loss_fn=loss_fn, optimizer=optimizer)
    validation_accuracy, validation_loss = test_validate(dataloader=validation_dataloader, model=model, loss_fn=loss_fn)
    print(f"Validation Error: \n Accuracy: {validation_accuracy:>0.3f}%, Avg loss: {validation_loss:>8f} \n")
    test_accuracy, test_loss = test_validate(dataloader=test_dataloader, model=model, loss_fn=loss_fn)
    print(f"Test Error: \n Accuracy: {test_accuracy:>0.3f}%, Avg loss: {test_loss:>8f} \n")

Epoch 0
-------------------------------
loss: 2.301747 [    0/50000]


C:\Users\ornel\PycharmProjects\cas_ai\venv\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss: 1.784618 [ 6400/50000]
loss: 1.675455 [12800/50000]
loss: 1.655410 [19200/50000]
loss: 1.644589 [25600/50000]
loss: 1.653207 [32000/50000]
loss: 1.630908 [38400/50000]
loss: 1.629176 [44800/50000]


KeyboardInterrupt: 